In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torchvision
from torchvision import models
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.transforms as transforms

In [2]:
# Data preprocessing

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'D:\\ML_face_rec'
train_small_dir = os.path.join(data_dir, 'train')
label_file = os.path.join(data_dir, 'train.csv')

In [3]:
# Create a custom dataset
class CustomImageFolder(Dataset):
    def __init__(self, data_dir, label_file, transform=None):
        self.data_dir = data_dir
        self.label_file = pd.read_csv(label_file)
        self.transform = transform

    def __len__(self):
        return len(self.label_file)

    
    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.label_file.iloc[idx, 1])
        image = Image.open(img_path).convert('RGB')
        label = self.label_file.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
    
        label = class_names.index(label)
        return image, label
    
    def __getitem__(self, idx):
        try:
            img_path = os.path.join(self.data_dir, self.label_file.iloc[idx, 1])
            image = Image.open(img_path).convert('RGB')
            label = self.label_file.iloc[idx, 2]
            if self.transform:
                image = self.transform(image)
            label = class_names.index(label)  
            return image, label
        except Exception as e:
            print(f"Error loading image: {self.label_file.iloc[idx, 1]}, {e}")

            return torch.zeros(3, 224, 224), 0

In [4]:
# Create datasets and dataloaders
train_dataset = CustomImageFolder(train_small_dir, label_file, transform=data_transforms['train'])
val_dataset = CustomImageFolder(train_small_dir, label_file, transform=data_transforms['val'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)


In [6]:
# Get class names
class_names = sorted(train_dataset.label_file['Category'].unique())
if len(class_names) != 100:
    raise ValueError("The number of unique categories in the CSV file should be 100.")

model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

In [7]:
# Training Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Training
num_epochs = 30
for epoch in range(num_epochs):
    # Training loop
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to my device
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to my device
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        val_correct += (preds == labels).sum().item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_correct/len(val_loader.dataset):.4f}')



c:\Users\123\anaconda3\envs\ML_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\123\anaconda3\envs\ML_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\123\anaconda3\envs\ML_env\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
c:\Users\123\anaconda3\envs\ML_env\lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1/30, Train Loss: 4.5044, Val Loss: 4.3049, Val Accuracy: 0.0395
Epoch 2/30, Train Loss: 4.1678, Val Loss: 3.9197, Val Accuracy: 0.0954
Epoch 3/30, Train Loss: 3.9218, Val Loss: 3.7723, Val Accuracy: 0.1253
Epoch 4/30, Train Loss: 3.7141, Val Loss: 3.3593, Val Accuracy: 0.2055
Epoch 5/30, Train Loss: 3.5264, Val Loss: 3.2206, Val Accuracy: 0.2377
Epoch 6/30, Train Loss: 3.3443, Val Loss: 3.0407, Val Accuracy: 0.2785
Epoch 7/30, Train Loss: 3.1690, Val Loss: 2.7224, Val Accuracy: 0.3557
Epoch 8/30, Train Loss: 3.0081, Val Loss: 2.6982, Val Accuracy: 0.3648
Epoch 9/30, Train Loss: 2.8643, Val Loss: 2.3260, Val Accuracy: 0.4507
Epoch 10/30, Train Loss: 2.7343, Val Loss: 2.2974, Val Accuracy: 0.4556
Epoch 11/30, Train Loss: 2.6223, Val Loss: 2.0423, Val Accuracy: 0.5243
Epoch 12/30, Train Loss: 2.5248, Val Loss: 2.0175, Val Accuracy: 0.5257
Epoch 13/30, Train Loss: 2.4470, Val Loss: 2.2335, Val Accuracy: 0.4749
Epoch 14/30, Train Loss: 2.3631, Val Loss: 1.8182, Val Accuracy: 0.5753
E

In [4]:
# Make predictions and generate submission file
model.eval()
predictions = []
test_dir = os.path.join(data_dir, 'test')
test_files = sorted(os.listdir(test_dir))

for file in test_files:
    img_path = os.path.join(test_dir, file)
    image = Image.open(img_path).convert('RGB')
    image = data_transforms['val'](image).unsqueeze(0).to(device, non_blocking=True)  # Move to GPU
    with torch.no_grad():
        outputs = model(image)
        _, pred = torch.max(outputs, 1)
        predictions.append(class_names[pred.item()])

submission_path = 'D:\\ML_face_rec\\submission.csv'
submission = pd.DataFrame({'Id': test_files, 'Category': predictions})
submission.to_csv(submission_path, index=False)
print(f"Submission file saved at: {submission_path}")

Submission file saved at: D:\ML_face_rec\submission.csv


In [10]:
# Make predictions and generate submission file
model.eval()
predictions = []
test_dir = os.path.join(data_dir, 'test')
test_files = sorted(os.listdir(test_dir))

for file in test_files:
    img_path = os.path.join(test_dir, file)
    image = Image.open(img_path).convert('RGB')
    image = data_transforms['val'](image).unsqueeze(0).to(device, non_blocking=True)  # Move to GPU
    with torch.no_grad():
        outputs = model(image)
        _, pred = torch.max(outputs, 1)
        predictions.append(class_names[pred.item()])

submission_path = 'D:\\ML_face_rec\\submission_new1_epoch30.csv'
file_names = [os.path.splitext(f)[0] for f in test_files] 
submission = pd.DataFrame({'Id': file_names, 'Category': predictions})
submission.to_csv(submission_path, index=False)
print(f"Submission file saved at: {submission_path}")

Submission file saved at: D:\ML_face_rec\submission_new1_epoch30.csv
